# Load and Preview Text Data
Import pandas, load the dataframe and examine the 'Text' column characteristics using basic statistics and sample visualization.

In [ ]:
#Importar librería para cargar el dataset y extraer columnas
%pip install -q pandas

import pandas as pd

# cargar el dataset desde la carpeta Data
data = pd.read_csv('Data/youtoxic_english_1000.csv')
df = data[['Text', 'IsToxic']]

print(df.head())
print(df.dtypes)
print("="*50)

print(f"Analisis descriptivo de la variable Text: \n{df['Text'].describe()}")
print("="*50)

print(f"Muestra aleatoria de la variable Text: \n{df['Text'].sample(10)}")

Note: you may need to restart the kernel to use updated packages.
                                                Text  IsToxic
0  If only people would just take a step back and...    False
1  Law enforcement is not trained to shoot to app...     True
2  \nDont you reckon them 'black lives matter' ba...     True
3  There are a very large number of people who do...    False
4  The Arab dude is absolutely right, he should h...    False
Text       object
IsToxic      bool
dtype: object
Analisis descriptivo de la variable Text: 
count              1000
unique              997
top       run them over
freq                  3
Name: Text, dtype: object
Muestra aleatoria de la variable Text: 
428    THANK YOU! right on.black people are going to ...
512    When did coming towards the officer is the sam...
107    This is a genuine failure of leadership by the...
47     This moron is talking about being peaceful?  T...
625    They just need to arrest all those protesters ...
942    i like how blac

In [ ]:
#Encontrar valores repetidos en la columna Text
duplicates = df[df.duplicated(['Text'], keep=False)]
print(f"Valores duplicados en la variable Text: \n{duplicates}")

Valores duplicados en la columna Text: 
              Text  IsToxic
592  RUN THEM OVER     True
642  run them over     True
657  run them over     True
677  run them over     True
699  RUN THEM OVER     True
